<a href="https://colab.research.google.com/github/minalang/study/blob/main/210727_tmdb%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%99%9C%EC%9A%A9_%EC%BD%98%ED%85%90%EC%B8%A0%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

credits: https://techblog-history-younghunjo1.tistory.com/115?category=924148

In [ ]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
movies = pd.read_csv('tmdb_5000_movies.csv', encoding='utf-8')
# 컨텐츠를 추천하는 시스템을 만들기 위해서
# 해당 영화들의 유사도 비교할 항목들만 추출
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
# genres라는 dict형태에서 'name' key의 value들만 가져오기
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
# keywords도 동일하게 수행
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
# 리스트에 담겨있는 genres, keywords를 공백 기준으로 문자열로 붙이기
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x))

In [ ]:
# 모든 변수가 텍스트로 이루어졌으니 Countvectorizer로 feature 벡터화시키기
# 경우에 따라 추가적으로 Lemmatization 추가해 토큰화 전용 함수 추가도 가능!

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
genres_vect = cnt_vect.fit_transform(movies_df['genres_literal'])
keywords_vect = cnt_vect.fit_transform(movies_df['keywords_literal'])

# 장르에 따른 영화별 코사인 유사도 추출
genre_sim = cosine_similarity(genres_vect, genres_vect)
# 3개만 유사도행렬값 추출해보기
print(genre_sim[:3])

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [ ]:
# argsort를 이용해서 유사도가 높은 영화들의 index 추출
genre_sim_idx = genre_sim.argsort()[::-1]

# 특정 영화를 기준으로 선정해서 유사도가 높은 영화를 반환하는 함수 생성
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    # 비교기준으로 선정할 특정 영화 선정
    title_movie = df[df['title'] == title_name]
    
    # 비교기준 영화의 index.values를 할당해 유사도 행렬에서 비교기준 영화에 해당하는 유사도 행렬값을 찾자!
    title_movie_idx = title_movie.index.values
    # 모든 영화끼리의 유사도 행렬에서 비교기준 영화에 대한 유사도행렬 찾아서 할당
    top_sim_idx = sorted_idx[title_movie_idx, :top_n]
    print(top_sim_idx)
    
    # top_sim_idx는 2차원 array이기 때문에 1차원 array로 변경
    # 왜냐하면 top_sim_idx로 movies_df의 index값으로 넣어서 유사한 영화추출할 것이기 때문
    top_sim_idx = top_sim_idx.reshape(-1,)
    similar_movie = df.iloc[top_sim_idx]
    
    return similar_movie
    
similar_movies = find_sim_movie(movies_df, genre_sim_idx, 'The Avengers')
print(similar_movies[['title','vote_average','vote_count']])

[[   0 2743 2749 2750 2751 2752 2755 2757 2760 2764]]
                      title  vote_average  vote_count
0                    Avatar           7.2       11800
2743   The Butterfly Effect           7.3        2060
2749         Child's Play 2           5.8         308
2750           No Good Deed           5.6         181
2751               The Mist           6.7        1399
2752             Ex Machina           7.6        4737
2755          Earth to Echo           5.7         290
2757  Letters from Iwo Jima           7.2         541
2760                   Room           8.1        2757
2764            Light It Up           6.6           7


In [ ]:
C = movies_df['vote_average'].mean()

# 투표횟수 중 60%이상의 횟수에 달하는 숫자
# 예를들어 총 투표횟수가 100과 1일때 m값은 매우 달라진다.
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R) + ( (m/(m+v)) * C)
    
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
 
# 가중 평점 변수를 추가해서 유사한 영화 찾아주는 함수 재정의
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_idx = title_movie.index.values
    
    similar_idx = sorted_idx[title_idx, :(top_n*2)]
    similar_idx = similar_idx.reshape(-1,)
    
    #자기 자신 영화 제외, boolean index기법 사용!
    similar_idx = similar_idx[similar_idx != title_idx]
    return df.iloc[similar_idx].sort_values(by=['weighted_vote'], ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_idx,
                               'The Avengers')
print(similar_movies[['title','vote_average','weighted_vote']])

                      title  vote_average  weighted_vote
2760                   Room           8.1       7.862312
2752             Ex Machina           7.6       7.490704
0                    Avatar           7.2       7.166301
2743   The Butterfly Effect           7.3       7.116008
2757  Letters from Iwo Jima           7.2       6.749914
2751               The Mist           6.7       6.572814
2783               The Call           6.6       6.484465
2787         Quest for Fire           7.1       6.321415
2742           The Shallows           6.2       6.179394
2778               Infamous           6.4       6.135104
